**Part 1:** Count visible trees
Scan an entire row/column starting from the outside and moving across the entire array. Track the tallest trees found and compare each to them in batch operations.

In [10]:
import numpy as np
np.set_printoptions(linewidth=1024, threshold=int(10e3))

def sanity_check(scan_buffer: np.ndarray):
    if -1 in scan_buffer:
        raise ValueError(f'Trees were not accounted for: {scan_buffer.argmin()}')

with open('../inputs/day8-input') as f:
    trees = np.array([[n for n in l.rstrip()] for l in f.readlines()], dtype=int)

# From the top: first axis, index increasing
tallest = np.full(trees.shape[1], -1)
visibles = np.full(trees.shape, False)
for i in range(trees.shape[0]):
    visibles[i] = trees[i] > tallest
    tallest = np.where(trees[i] > tallest, trees[i], tallest)
sanity_check(tallest)

# From the bottom
tallest[:] = -1
for i in range(trees.shape[0] - 1, -1, -1):
    visibles[i] = np.logical_or(visibles[i], trees[i] > tallest)
    tallest = np.where(trees[i] > tallest, trees[i], tallest)
sanity_check(tallest)

# From the left: second axis, index increasing
tallest.reshape((1, trees.shape[0]))
tallest[:] = -1
for i in range(trees.shape[0]):
    visibles[:, i] = np.logical_or(visibles[:, i], trees[:, i] > tallest)
    tallest = np.where(trees[:, i] > tallest, trees[:, i], tallest)
sanity_check(tallest)

# From the right
tallest[:] = -1
for i in range(trees.shape[0] - 1, -1, -1):
    visibles[:, i] = np.logical_or(visibles[:, i], trees[:, i] > tallest)
    tallest = np.where(trees[:, i] > tallest, trees[:, i], tallest)
sanity_check(tallest)

print(str(visibles).replace(' ', '').replace('False', 'x').replace('True', 'O'))
print(f'{visibles.sum()} visible trees')

[[OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO]
[OOOxxOxOOxxxxxxxOOOxOxOxxOOOxxOxOxOOOxOxxOxOOxOxOOOxxOxOxxOOxOOOOxxxxxxxOxOOOOOxOxOxxxxOOxxOOxOxxOO]
[OOxxxOxxOOxxOOxxxxxxxxxxxOxxxxxxxxxOOOxxOxxxxOxxxxxxOOOxxxOxxxOOOxxxOxOxxxxxOOOxOxxxxxxxOOxxOxxOxxO]
[OOxxxxxOxxxxOxxxxxxxOxxxxxxxOxxxxOxOxOxxxxxxxxxOxOxxxxxxxxxxxOOxxxOxxxOxxxxxxOxxxxxxxxxxxxxxxOOxxOO]
[OxxxxxxxxxxxxOxxOxxxxOxOxxxxxxxOxxxOxxxxxxxxxxxxxxxOxxxxxOxxxxxxxOxOxxOxxxxxxxxxOxxxxxxOxxxxOxxxOOO]
[OxxxOxxOxOxxOxOxxxOxxxxxxxxOxxOxxxxxxOxxxxOxOOxxxxxxxxxxxxxxxOxxxxxxxxxOxxxxxxxxxxxxxOOxxOxxxxxxOxO]
[OOxOxxxxxOxxxxxOxxxxxxxxxxOxOxxxOxxxxxxxxxxxxxxxxOxxxxxxxxxxxxxxxxxOOOxxOOOxxxxxxOxxxxxxxxOxOxxxxOO]
[OxOxxxxxxxOOxxxxOOxxxxxxxxxOxxxxxxOxxxxxxxxxxxxxxxOxxOxxxxOxxxxxxxxOxxxOxxxxxxxxxxxOxxOxxxxxxOxxxxO]
[OOxxOxOxxxxxxxxxxxOxxxxxxxxxxxxxOxxxxxxxxxxxxxxxxxxxxOxxOxxxxxxxxxxxxxxxxxOxxxxxxxOxxxxxxxxxxxxxxOO]
[OOOxxxxxxxxxxOxxxxxxxOxxxOxxxxxxxxxxOxOxxxxxxxxxxOxxxxxxxxxxOOxxxxxxxxxOxxxxOxOx

**Part 2:** Find the tree with the highest scenic score
Rather than scan from a row/col as in pt 1, scan from a point moving out toward the edges. All dimensions drop by 1 from the earlier code. That's the big conceptual change.

In [14]:
np.set_printoptions(linewidth=1024, edgeitems=8, threshold=int(1e3))
scores = np.full(trees.shape, -1)

def scan_lr(coords: tuple, direction: str) -> int:
    if direction == 'l':
        start, stop, step = (coords[1] - 1, -1, -1)
    elif direction == 'r':
        start, stop, step = (coords[1] + 1, trees.shape[1], 1)
    else:
        raise ValueError(f'{direction} is not an allowed direction.')

    for _i in range(start, stop, step):
        if trees[coords[0], _i] >= trees[coords]:
            return _i  # Line of sight has been broken
    return stop + 1 if direction == 'l' else stop - 1  # If it ran off the edge

def scan_ud(coords: tuple, direction: str) -> int:
    if direction == 'u':
        start, stop, step = (coords[0] - 1, -1, -1)
    elif direction == 'd':
        start, stop, step = (coords[0] + 1, trees.shape[0], 1)
    else:
        raise ValueError(f'{direction} is not an allowed direction.')

    for _i in range(start, stop, step):
        if trees[_i, coords[1]] >= trees[coords]:
            return _i  # Line of sight has been broken
    return stop + 1 if direction == 'u' else stop - 1  # If it ran off the edge

for t in np.ndindex(trees.shape):
    l = t[1] - scan_lr(t, 'l')
    r = scan_lr(t, 'r') - t[1]
    u = t[0] - scan_ud(t, 'u')
    d = scan_ud(t, 'd') - t[0]
    sum_score = l + r + u + d  # for debugging
    scores[t] = l * r * u * d

sanity_check(scores)
print(scores)
print(f'Max score: {scores.max()}')

[[  0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   0]
 [  0   1   6  10   1   4   1   4 ...  36   6   1   4   2   1   2   0]
 [  0   1   3   1   1  20   4   1 ...   2  36   4   1  72   1   2   0]
 [  0  45   1   2   6   2  25  42 ...   1   2   3 960   1   6   1   0]
 [  0   1  90   6   1   1  12  20 ...   1  72   9   1   8  12   8   0]
 [  0   1   1   1  60   4   1 210 ...  48   1   1  60   1   4   1   0]
 [  0  12   2 216   2  16   1   1 ...   1  24   1   8  12   5   4   0]
 [  0   1  18   1   1  30   1   4 ...  12   4  35   1   6   2   1   0]
 ...
 [  0   2   1 432   3  10   1   8 ...   2  12  30   1   4   1   8   0]
 [  0  18 288   1 672   2  24   1 ...   1  18   2   4   8   2   2   0]
 [  0   1   2  24   3   1   8  80 ...   2   4  50   1   2   9   1   0]
 [  0   2   1  60   6   8   1   3 ...   1  10   1  72   8   3 156   0]
 [  0   1   2   1   3   1   1  28 ...   6  54  16   1   2   1  12   0]
 [  0   1   1   1   8  20   1   4 ...   2   4   1 144   3   1   1   0]
 